In [ ]:
# https://medium.com/data-hackers/engenharia-de-features-transformando-dados-categ%C3%B3ricos-em-dados-num%C3%A9ricos-e5d3991df715

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import matthews_corrcoef, make_scorer, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("train.csv") 
# apagando colunas que não auxiliam na predição
df = df.drop(["ano", "sequencial_candidato", "nome"], axis=1)
# lidando com as variaveis categoricas
df = pd.get_dummies(df, columns = ["uf", "partido", "cargo", "sexo", "grau", "estado_civil", "ocupacao"]) 
# separando dados de treino e teste
train, test = train_test_split(df, train_size=0.8) # separação entre treino e teste  

train_y = train[["situacao"]]
train_X = train.drop(["situacao"], axis = 1)

test_y = test[["situacao"]]
test_X = test.drop(["situacao"], axis = 1)

le = LabelEncoder()
le.fit(train_y.situacao)
train_y = le.transform(train_y.situacao)

le = LabelEncoder()
le.fit(test_y.situacao)
test_y = le.transform(test_y.situacao)


In [3]:
df.groupby("situacao").size()

situacao
eleito        1026
nao_eleito    6596
dtype: int64

P: Há desbalanceamento das classes (isto é, uma classe tem muito mais instâncias que outra)? 

R: Sim, Temos um dataset desbalanceado com mais classes do tipo "não eleito". mais ou menos 5 para 1, do que para o "eleito".

P: Em que proporção? 

R: Mais de 5 para 1, ou seja a cada 1 eleito temos quase 6 não eleitos.

P: Quais efeitos colaterais o desbalanceamento de classes pode causar no classificador?

R: Alguns algoritmos de ML tem dificuldade em em induzir um bom modelo a partir de conjutos de dados desbalanceados. Se classificarmos com dados da classe desbalanceados, os algoritmos podem não diferenciar  a classe minoritária das demais categorias, acreditando que estão agregando resultado devido à aparente alta acurácia. Essa falta de diferenciação pode ocasionar problemas visto que geralmente a classe minoritária é o cerne da questão, como no nosso caso, quem foi eleito.

P: Como você poderia tratar isso? (10 pt.)

R: Pode ser utilizado as técnicas de "Reestruturação dos dados". Dentro das ténicas de "Reestruturação de dados" pode ser feito o "Undersamplig", que é reduzir a distribuição dos dados das observações da classe majoritária, para tentar igualar a quantidade. Isso pode ser feito com "Random Undersamplig", que é a retirada aleatória de observações da classe majoritária, ou "Fusão" que é unir duas ou mais observações da classe majoritária para uma menor perda de informação. Outra forma é o Oversampling consite em criar novas observações da classe minoritária com o objetivo de igualar a proporção das categorias. Outra opção é escolher um algoritmo mais resilente, coletar mais dados e usar modelos penalizados ou utilizar outras métricas para treino.

Fonte: https://medium.com/turing-talks/dados-desbalanceados-o-que-s%C3%A3o-e-como-evit%C3%A1-los-43df4f49732b


2 - Treine: 

2.1 - um modelo de regressão logística, 

2.2 - uma árvore de decisão, 

2.3 - um modelo de adaboost, 

2.4 - um modelo de random forest e 

2.5 - um modelo de gradient boosting. 

2.6 - Tune esses modelos usando validação cruzada e controle overfitting se necessário, considerando as particularidades de cada modelo.  (10 pts.)

Lidando com as variáveis Dummies

Regressao Logistica

In [5]:
clf = LogisticRegression()
clf.fit(train_X, train_y)
scoring = make_scorer(matthews_corrcoef)
y_pred = clf.predict(test_X)

print("MCC:", matthews_corrcoef(test_y, y_pred)) # métrica boa para avaliar modelos em dados desbalencados
print("F1:", f1_score(test_y, y_pred)) # O F1 só é baixo em dados desbalanceados se a classe minoritária tiver label 1
print("ACC:", accuracy_score(test_y, y_pred)) # Paradoxo da acurrácia

MCC: 0.5269897492365765
F1: 0.9448760636330004
ACC: 0.9022950819672131


c:\Users\gigi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [8]:
clf = svm.SVC()
clf.fit(train_X, train_y)
scoring = make_scorer(matthews_corrcoef)
y_pred = clf.predict(test_X)

print("MCC: ", matthews_corrcoef(test_y , y_pred))
print("F1:", f1_score(test_y , y_pred))
print("ACC:", accuracy_score(test_y , y_pred))

MCC:  0.5749912854492889
F1: 0.9477221807318894
ACC: 0.9081967213114754


In [10]:
clf = svm.SVC()
scoring = {"mcc" : make_scorer(matthews_corrcoef), "f1":"f1", "acc": "accuracy"}
clf.fit(train_X, train_y)
result_ros = cross_validate(clf, train_X, train_y, cv = 5, scoring=scoring)
print("MCC:", round(result_ros['test_mcc'].mean(), 4))
print("F1:", round(result_ros['test_f1'].mean(), 4))
print("ACC:", round(result_ros['test_acc'].mean(), 4))

MCC: 0.5871
F1: 0.9489
ACC: 0.9103


In [ ]:
#from sklearn import tree
#clf = tree.DecisionTreeClassifier()
#clf.fit(df_train_X, labels_train_Y)
